In [5]:
from sqlalchemy.orm import Session
import sys
sys.path.append('..')
from app import models
from app.database import engine,SessionLocal
from pprint import pprint
import pandas as pd
from fastapi.encoders import jsonable_encoder
import random
from IPython.display import HTML

In [6]:
db = SessionLocal()

# Logic to create the quiz for a single user

Example userId = 1

In [39]:
user_id=1
steckbrief_id=1

## Used tables

In [40]:
display(HTML(f"<h3>Table: User</h3><p>user_id =1</p>"))
display(HTML(pd.read_sql(db.query(models.User).filter(models.User.user_id==user_id).statement,db.bind).to_html(index=False)))

display(HTML(f"<h3>Table: Steckbrief</h3><p>steckbrief_id =1</p>"))
display(HTML(pd.read_sql(db.query(models.Steckbrief).filter(models.Steckbrief.steckbrief_id==steckbrief_id).statement,db.bind).to_html(index=False)))

display(HTML(f"<h3>Table: Steckbrief_Fragen</h3><p>steckbrief_id =1</p>"))
display(HTML(pd.read_sql(db.query(models.Steckbrief_Frage).filter(models.Steckbrief_Frage.steckbrief_id==steckbrief_id).statement,db.bind).to_html(index=False)))

display(HTML(f"<h3>Table: Frage</h3>"))
display(HTML(pd.read_sql(db.query(models.Frage).statement,db.bind).to_html(index=False)))

display(HTML(f"<h3>Table: Dummy_Antwort</h3>"))
display(HTML(pd.read_sql(db.query(models.Dummy_Antwort).statement,db.bind).to_html(index=False)))

user_id,abteilung_id,steckbrief_id,beruf,vorname,nachname,geburtsdatum
1,1,1,Grafiker,Marco,Gerken,1993-05-12


steckbrief_id,freigegeben
1,True


steckbrief_id,frage_id,antwort
1,1,Python
1,2,42
1,3,Paulaner Spezi
1,4,Spaghetti


frage_id,frage
1,Was ist deine Lieblings-Programmiersprache?
2,Was ist der Sinn des Lebens?
3,Was ist dein Lieblingsgetränk?
4,Welche Nudelsorte isst du am liebsten?


dummy_antwort_id,frage_id,antwort
1,1,Java
2,1,R
3,1,Rust
4,1,Scala
5,1,Kotlin
6,1,Swift
7,2,Leben
8,2,Keine Ahnung
9,2,Genießen
10,2,Den ganzen Tag Hamburger essen


## Step 1 - Get questions and answers as single table

Join Tables User + Steckbrief + Steckbrief_Frage + Frage

In [61]:
query= db.query(models.User,models.Steckbrief,models.Steckbrief_Frage,models.Frage)\
.with_entities(models.Steckbrief.steckbrief_id,models.Frage.frage_id,models.Frage.frage,models.Steckbrief_Frage.antwort)\
.filter(models.User.steckbrief_id==models.Steckbrief.steckbrief_id,
models.Steckbrief.steckbrief_id==models.Steckbrief_Frage.steckbrief_id,
models.Steckbrief_Frage.frage_id==models.Frage.frage_id,
models.User.user_id==user_id)

display(HTML(f"<h3>SQL</h3>"))
print(query)

display(HTML(f"<h3>Result</h3>"))

df_frage_antwort=pd.read_sql(query.statement,db.bind)[["frage_id","frage","antwort"]]

display(HTML(df_frage_antwort.to_html(index=False)))

SELECT steckbrief.steckbrief_id AS steckbrief_steckbrief_id, frage.frage_id AS frage_frage_id, frage.frage AS frage_frage, steckbrief_frage.antwort AS steckbrief_frage_antwort 
FROM steckbrief, frage, steckbrief_frage, user 
WHERE user.steckbrief_id = steckbrief.steckbrief_id AND steckbrief.steckbrief_id = steckbrief_frage.steckbrief_id AND steckbrief_frage.frage_id = frage.frage_id AND user.user_id = ?


frage_id,frage,antwort
1,Was ist deine Lieblings-Programmiersprache?,Python
2,Was ist der Sinn des Lebens?,42
3,Was ist dein Lieblingsgetränk?,Paulaner Spezi
4,Welche Nudelsorte isst du am liebsten?,Spaghetti


## Step 2 - Get matching dummy answers

In [71]:
query = db.query(models.Dummy_Antwort).filter(models.Dummy_Antwort.frage_id.in_(df["frage_id"].tolist())) 
    
dummy_answers = pd.read_sql(db.query(models.Dummy_Antwort).filter(models.Dummy_Antwort.frage_id.in_(df["frage_id"].tolist())).statement, db.bind)

display(HTML(f"<h3>SQL</h3>"))
print(query)

display(HTML(f"<h3>Result - Example</h3>"))
display(HTML(dummy_answers.head(10).to_html(index=False)))
print("...")

SELECT dummy_antwort.dummy_antwort_id AS dummy_antwort_dummy_antwort_id, dummy_antwort.frage_id AS dummy_antwort_frage_id, dummy_antwort.antwort AS dummy_antwort_antwort 
FROM dummy_antwort 
WHERE dummy_antwort.frage_id IN (?, ?, ?, ?)


dummy_antwort_id,frage_id,antwort
1,1,Java
2,1,R
3,1,Rust
4,1,Scala
5,1,Kotlin
6,1,Swift
7,2,Leben
8,2,Keine Ahnung
9,2,Genießen
10,2,Den ganzen Tag Hamburger essen


...


## Step 3 - Join dummy answers to questions and convert them as list

In [74]:
df_merged=(df_frage_antwort.merge(dummy_answers.groupby('frage_id')['antwort']\
                                  .apply(list),how="left",on="frage_id",suffixes=["_original","_extra"]))

display(HTML(df_merged.to_html(index=False)))

frage_id,frage,antwort_original,antwort_extra
1,Was ist deine Lieblings-Programmiersprache?,Python,"[Java, R, Rust, Scala, Kotlin, Swift]"
2,Was ist der Sinn des Lebens?,42,"[Leben, Keine Ahnung, Genießen, Den ganzen Tag Hamburger essen]"
3,Was ist dein Lieblingsgetränk?,Paulaner Spezi,"[Kölsch, Alt, Cola, Radler, Költ]"
4,Welche Nudelsorte isst du am liebsten?,Spaghetti,"[Penne, Cannelloni, Farfalle, Gnocchi, Fettuccine]"


## Step 4 - Create question catalog from joined table

In [101]:
quiz_dict=df_merged.T.to_dict().values()
quiz_list=[]

## Example of one iteration

In [123]:
for item in list(quiz_dict)[:1]:
    row = dict(item)
    display(HTML("<h4>Before</h4>"))
    pprint(row)
    catalog=[]
    catalog.append(row["antwort_original"])
    display(HTML("<h4>Add original answer to catalog</h4>"))
    pprint(f"Catalog: {catalog}")
    
    display(HTML("<h4>Shuffle extra answers</h4>"))
    pprint(f"Before: {row['antwort_extra']}")
    random.shuffle(row["antwort_extra"])
    pprint(f"After:  {row['antwort_extra']}")
    
    display(HTML("<h4>Append 3 answers to catalog and shuffle again</h4>"))
    catalog+=row["antwort_extra"][:3]
    random.shuffle(catalog)
    pprint(f"Final catalog: {catalog}")
    row["catalog"]=catalog
    del row['antwort_extra']
    
    display(HTML("<h4>Resultobject for one question</h4>"))
    pprint(row)
    

{'antwort_extra': ['Java', 'Rust', 'Kotlin', 'R', 'Scala', 'Swift'],
 'antwort_original': 'Python',
 'frage': 'Was ist deine Lieblings-Programmiersprache?',
 'frage_id': 1}


"Catalog: ['Python']"


"Before: ['Java', 'Rust', 'Kotlin', 'R', 'Scala', 'Swift']"
"After:  ['R', 'Java', 'Rust', 'Scala', 'Swift', 'Kotlin']"


"Final catalog: ['Python', 'R', 'Java', 'Rust']"


{'antwort_original': 'Python',
 'catalog': ['Python', 'R', 'Java', 'Rust'],
 'frage': 'Was ist deine Lieblings-Programmiersprache?',
 'frage_id': 1}
